In [19]:
import numpy as np
import pandas as pd
import re
import xgboost
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score,StratifiedKFold,GridSearchCV
#for rafi to run

In [2]:
work_train= {} 
work_test = {}
pre_process=["bio.csv","wind_tfidf_100.csv","full_tfidf_100.csv","w2v_300.csv"]
path="./window/checkpoints_databases/"
for f in pre_process:
    work_train[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_train_"+f)
    work_test[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_test_"+f)

In [3]:
#External features + genes tsvd
feat_train_1=pd.read_csv("./window/w_meta_features/meta_train_l1l2.csv") #Gene tsvd
feat_test_1=pd.read_csv("./window/w_meta_features/meta_test_l1l2.csv") 
feat_train_1=feat_train_1.drop("ID",axis=1)
feat_test_1=feat_test_1.drop("ID",axis=1)
feat_train_2=pd.read_csv("./window/w_meta_features/new_working_train_ext.csv")#External tsvd
feat_test_2=pd.read_csv("./window/w_meta_features/new_working_test_ext.csv")

In [4]:
train=pd.concat((work_train["bio"],work_train["wind_tfidf_100"],work_train["full_tfidf_100"],work_train["w2v_300"],feat_train_1,feat_train_2),axis=1)

In [5]:
test=pd.concat((work_test["bio"],work_test["wind_tfidf_100"],work_test["full_tfidf_100"],work_test["w2v_300"],feat_test_1,feat_test_2),axis=1)

In [6]:
new_train=pd.read_csv("window/checkpoints_databases/new_working_train.csv")
new_test=pd.read_csv("window/checkpoints_databases/new_working_test.csv")
y=new_train.Class - 1
ID_train=new_train.ID
ID_test=new_test.ID
del new_train, new_test


In [15]:
kf=StratifiedKFold(n_splits=5, random_state=26, shuffle=True)

In [ ]:
X_train=np.array(train)
X_test=np.array(test)

In [16]:
clf_xgb=XGBClassifier(max_depth=5, objective="multi:softprob",seed=26)
h=cross_val_score(clf_xgb,np.array(train),y,cv=kf,n_jobs=-1,scoring="neg_log_loss")
print("mean : "+str(h.mean()),
         "std : "+name+" "+str(h.std()))

NameError: name 'name' is not defined

In [ ]:
clf_xgb=XGBClassifier(max_depth=3, objective="multi:softprob",seed=26)
param_test= {
    "max_depth" : [3,5,7],
    "min_child_weight" : [1,3,5],
    "subsample":[0.8,0.9,1],
    "colsample_bytree":[0.8,0.9,1]
}
gsearch=GridSearchCV(estimator=clf_xgb, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
gsearch.fit(np.array(train),y)
print(gsearch.grid_scores_)
print(gsearch.best_params_)
print(gsearch.best_score_)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 22.4min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 127.3min


In [ ]:
def model_gen(X,X_test,y,classifier):
    kf = StratifiedKFold(n_splits=5, random_state=26, shuffle=True)
    fold = 0
    y_test=0
    for train_index, test_index in kf.split(X, y):
        
        fold += 1

        X_train, X_valid    = X[train_index],   X[test_index]
        y_train, y_valid    = y[train_index],   y[test_index]

        print("Fold", fold, X_train.shape, X_valid.shape)

        clf=classifier
        clf.fit(X_train,y_train)
        p_test = clf.predict_proba(X_test)
        y_test += p_test/5
    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("../final_scores/new_backup_sub.csv",index=False)
    

In [ ]:
clf = XGBClassifier()
model_gen(X_train,X_test,y,clf)